**Data** **fields**

Most of the fields are self-explanatory. The following are descriptions for those that aren't.

**Id** - an Id that represents a (Store, Date) duple within the test set

**Store** - a unique Id for each store

 **Sales** - the turnover for any given day (this is what you are predicting)

**Customers** - the number of customers on a given day
**Open** - an indicator for whether the store was open: 0 = closed, 1 = open

**StateHoliday** - indicates a state holiday. Normally all stores, with few         exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None

**SchoolHoliday** - indicates if the (Store, Date) was affected by the closure of public schools

**StoreType** - differentiates between 4 different store models: a, b, c, d

**Assortment** - describes an assortment level: a = basic, b = extra, c = extended

**CompetitionDistance** - distance in meters to the nearest competitor store

**CompetitionOpenSince[Month/Year]** - gives the approximate year and month of the time the nearest competitor was opened

**Promo** - indicates whether a store is running a promo on that day

**Promo2** - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating

**Promo2Since[Year/Week]** - describes the year and calendar week when the store started participating in Promo2

**PromoInterval** - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store


In [1]:
import warnings
warnings.filterwarnings("ignore")

# loading packages
# basic + dates 
import numpy as np
import pandas as pd
import datetime
import pickle

#SKlearn Modelling and data transform
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error


In [2]:
train = pd.read_csv("https://iyanu2.blob.core.windows.net/unzipped/rossmann-store-sales/train.csv")
test = pd.read_csv("https://iyanu2.blob.core.windows.net/unzipped/rossmann-store-sales/test.csv")
store = pd.read_csv("https://iyanu2.blob.core.windows.net/unzipped/rossmann-store-sales/store.csv")

### Merging the data with Store details

In [3]:
train = pd.merge(train, store, on='Store')
test = pd.merge(test, store, on='Store')

In [4]:
train = train.loc[train.Open != 0]
train = train.loc[train.Sales > 0].reset_index(drop=True)

In [5]:
train['PromoInterval']=train['PromoInterval'].fillna(0)
test['PromoInterval']=test['PromoInterval'].fillna(0)

In [6]:
def create_feature(dataset):
    """
    The create feature function would help create features using the date
    column and also other features 
    """
    
    #i would map some categorical variable to numerical if
    #that variable present it is replaced
    mappings = {'0':0, 'a':1, 'b':2, 'c':3, 'd':4}
    dataset.StoreType.replace(mappings, inplace=True)
    dataset.Assortment.replace(mappings, inplace=True)
    dataset.StateHoliday.replace(mappings, inplace=True)

    #converting some categorical variables to object
    categ =['DayOfWeek','Open','Promo','StateHoliday','SchoolHoliday']
    for i in categ:
        dataset = dataset.astype({i:'object'})
    
    
    #convert date column to datetime
    dataset['Date']= pd.to_datetime(dataset.Date)
    #Feature creation
    dataset['Year'] = dataset.Date.dt.year
    dataset['Month'] = dataset.Date.dt.month
    dataset['Day'] = dataset.Date.dt.day
    dataset['DayOfWeek'] = dataset.Date.dt.dayofweek
    dataset['WeekOfYear'] = dataset.Date.dt.weekofyear
    dataset['CompetitionOpen'] = 12*(dataset.Year-dataset.CompetitionOpenSinceYear
                                     ) + (dataset.Month-dataset.CompetitionOpenSinceMonth)
    dataset['PromoOpen'] = 12*(dataset.Year-dataset.Promo2SinceYear) + (
        dataset.WeekOfYear-dataset.Promo2SinceWeek)/4.0
    dataset['CompetitionOpen'] = dataset.CompetitionOpen.apply(
        lambda x: x if x > 0 else 0)        
    dataset['PromoOpen'] = dataset.PromoOpen.apply(
        lambda x: x if x > 0 else 0)
    
    month2str = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', 7:'Jul',
                 8:'Aug', 9:'Sept', 10:'Oct', 11:'Nov', 12:'Dec'}
    dataset['monthStr'] = dataset.Month.map(month2str)
    
    dataset.loc[dataset.PromoInterval==0, 'PromoInterval'] = ''
    dataset['IsPromoMonth'] = 0
    for interval in dataset.PromoInterval.unique():
        if interval != '':
            for month in interval.split(','):
                dataset.loc[(dataset.monthStr == month) & (dataset.PromoInterval == interval), 'IsPromoMonth'] = 1
   
 
    dataset = dataset.sort_values('Date')
    dataset = dataset.reset_index(drop=True)
    return dataset


In [7]:
train = create_feature(train)
test = create_feature(test)

In [8]:
test =test.sort_values(['Store',"Date"])

In [9]:
test

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Year,Month,Day,WeekOfYear,CompetitionOpen,PromoOpen,monthStr,IsPromoMonth
842,40233,1,5,2015-08-01,1,0,0,1,3,1,1270.0,9.0,2008.0,0,NaN,NaN,,2015,8,1,31,83.0,0.00,Aug,0
1223,39377,1,6,2015-08-02,0,0,0,1,3,1,1270.0,9.0,2008.0,0,NaN,NaN,,2015,8,2,31,83.0,0.00,Aug,0
1977,38521,1,0,2015-08-03,1,1,0,1,3,1,1270.0,9.0,2008.0,0,NaN,NaN,,2015,8,3,32,83.0,0.00,Aug,0
2634,37665,1,1,2015-08-04,1,1,0,1,3,1,1270.0,9.0,2008.0,0,NaN,NaN,,2015,8,4,32,83.0,0.00,Aug,0
4252,36809,1,2,2015-08-05,1,1,0,1,3,1,1270.0,9.0,2008.0,0,NaN,NaN,,2015,8,5,32,83.0,0.00,Aug,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36866,4280,1115,6,2015-09-13,0,0,0,0,4,3,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec",2015,9,13,37,0.0,39.75,Sept,1
38207,3424,1115,0,2015-09-14,1,1,0,0,4,3,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec",2015,9,14,38,0.0,40.00,Sept,1
39120,2568,1115,1,2015-09-15,1,1,0,0,4,3,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec",2015,9,15,38,0.0,40.00,Sept,1
39428,1712,1115,2,2015-09-16,1,1,0,0,4,3,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec",2015,9,16,38,0.0,40.00,Sept,1


In [10]:
train.to_csv("train.csv")

In [ ]:
y = train.pop('Sales')
train = train.drop(['Customers'], axis=1)
test = test.drop(['Id'], axis=1)

In [ ]:
 y = np.log1p(y)

In [ ]:
  x_train, x_val, y_train, y_val = train_test_split(train, y, test_size = .20, random_state = 0)

In [ ]:
numeric_features = train._get_numeric_data().columns
categorical_features = train.select_dtypes(include=['object']).columns

In [ ]:
 """
    The preprocess function takes as primary argument the d 
    and peform the following stepwise transformations to it:
    
    1. impute missing values of numerical and categorical columns 
    using median and constant values respectively
    
    2. scales dataset using the RobustScaler (robust to outlier values present in this dataset)
    
    3. Encodes categorical values to numerical values
"""
  
# build pipeline to preprocess
numeric_transformer = Pipeline([('imputer', SimpleImputer(strategy='median')),
                                ('scaler', RobustScaler())])
# for categorical variable create new category called missing
categorical_transformer = Pipeline([('imputer', SimpleImputer(strategy='constant',
                                                              fill_value='missing')),
                                    ('onehot', OneHotEncoder(handle_unknown='ignore'))])


features  = Pipeline([('features', ColumnTransformer([
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)]))
])
model = Pipeline([('feature', features ),
                ('regressor', RandomForestRegressor())
])


In [ ]:
model.fit(x_train, y_train)

Pipeline(memory=None,
         steps=[('feature',
                 Pipeline(memory=None,
                          steps=[('features',
                                  ColumnTransformer(n_jobs=None,
                                                    remainder='drop',
                                                    sparse_threshold=0.3,
                                                    transformer_weights=None,
                                                    transformers=[('num',
                                                                   Pipeline(memory=None,
                                                                            steps=[('imputer',
                                                                                    SimpleImputer(add_indicator=False,
                                                                                                  copy=True,
                                                                                            

Our model Accuracy for our test data

In [ ]:
y_pred = model.predict(x_val)
print("Mean Absolute Error: ", mean_absolute_error(y_val, y_pred).round(4))
print("Mean Squared Error: ", mean_squared_error(y_val, y_pred).round(4))

Mean Absolute Error:  0.0845
Mean Squared Error:  0.0148


In [ ]:
filename = '/content/drive/My Drive/rossman-stores/naive_rf.pkl'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
predictions = model.predict(test)


In [ ]:
 back = np.expm1(predictions)

In [ ]:
back

array([4850.17674839, 4874.91580274, 5735.83253395, ..., 7196.82131673,
       7079.50479042, 7196.56574006])

In [ ]:
test

,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Year,Month,Day,WeekOfYear,CompetitionOpen,PromoOpen,monthStr,IsPromoMonth
842,1,5,2015-08-01,1,0,0,1,3,1,1270.0,9.0,2008.0,0,NaN,NaN,,2015,8,1,31,83.0,0.00,Aug,0
1223,1,6,2015-08-02,0,0,0,1,3,1,1270.0,9.0,2008.0,0,NaN,NaN,,2015,8,2,31,83.0,0.00,Aug,0
1977,1,0,2015-08-03,1,1,0,1,3,1,1270.0,9.0,2008.0,0,NaN,NaN,,2015,8,3,32,83.0,0.00,Aug,0
2634,1,1,2015-08-04,1,1,0,1,3,1,1270.0,9.0,2008.0,0,NaN,NaN,,2015,8,4,32,83.0,0.00,Aug,0
4252,1,2,2015-08-05,1,1,0,1,3,1,1270.0,9.0,2008.0,0,NaN,NaN,,2015,8,5,32,83.0,0.00,Aug,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36866,1115,6,2015-09-13,0,0,0,0,4,3,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec",2015,9,13,37,0.0,39.75,Sept,1
38207,1115,0,2015-09-14,1,1,0,0,4,3,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec",2015,9,14,38,0.0,40.00,Sept,1
39120,1115,1,2015-09-15,1,1,0,0,4,3,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec",2015,9,15,38,0.0,40.00,Sept,1
39428,1115,2,2015-09-16,1,1,0,0,4,3,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec",2015,9,16,38,0.0,40.00,Sept,1


In [ ]:
sub= test[['Store','Date']]

In [ ]:
sub['Prediction'] = back

In [ ]:
sub.to_csv("/content/drive/My Drive/rossman-stores/prediction.csv")